<a href="https://colab.research.google.com/github/Samin-Sadaf7/Image_works/blob/main/yolov5_objectDetection_with_roboflow_dataset.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>



```
# The link to follow on : https://www.youtube.com/watch?v=lJT_KRFLLRY
#  the colab repository to follow along: https://colab.research.google.com/drive/1ntAL_zI68xfvZ4uCSAF6XT27g0U4mZbW#scrollTo=VHS_o3KGIyXm
```



In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import zipfile
import os

def extract_zip(zip_path, extract_to):
    if not os.path.exists(extract_to):
        os.makedirs(extract_to)
    with zipfile.ZipFile(zip_path, 'r') as zip_ref:
        zip_ref.extractall(extract_to)

In [3]:
import os
import torch
from IPython.display import Image, clear_output
print('PyTorch %s %s' % (torch.__version__, torch.cuda.get_device_properties(0) if torch.cuda.is_available() else 'CPU'))

PyTorch 2.4.0+cu121 _CudaDeviceProperties(name='Tesla T4', major=7, minor=5, total_memory=15102MB, multi_processor_count=40)


In [ ]:
%cd /content

/content


In [4]:
!git clone https://github.com/ultralytics/yolov5  # clone
%cd yolov5
%pip install -qr requirements.txt  # install
import torch
import utils
display = utils.notebook_init()  # checks

YOLOv5 🚀 v7.0-364-g4d39325f Python-3.10.12 torch-2.4.0+cu121 CUDA:0 (Tesla T4, 15102MiB)


Setup complete ✅ (2 CPUs, 12.7 GB RAM, 32.9/78.2 GB disk)


In [ ]:
%cd ..

/content


In [ ]:
%cd /content/drive/MyDrive/Dataset- -Conference

/content/drive/MyDrive/Dataset- -Conference


In [ ]:
import shutil
import os

def copy_folder_contents(source_folder, destination_folder):
    # Ensure the destination folder exists
    if not os.path.exists(destination_folder):
        os.makedirs(destination_folder)

    # Copy each file and folder from the source to the destination
    for item in os.listdir(source_folder):
        source_path = os.path.join(source_folder, item)
        destination_path = os.path.join(destination_folder, item)

        if os.path.isdir(source_path):
            # Recursively copy directories
            shutil.copytree(source_path, destination_path, dirs_exist_ok=True)
        else:
            # Copy files
            shutil.copy2(source_path, destination_path)

In [ ]:
import os
import shutil

def sync_folders(folder1, folder2):
    # Get the list of files in both folders
    files_in_folder1 = set(os.listdir(folder1))
    files_in_folder2 = set(os.listdir(folder2))

    # Find files present in folder1 but not in folder2
    unique_to_folder1 = files_in_folder1 - files_in_folder2
    # Find files present in folder2 but not in folder1
    unique_to_folder2 = files_in_folder2 - files_in_folder1

    # Copy unique files from folder1 to folder2
    for file in unique_to_folder1:
        src = os.path.join(folder1, file)
        dst = os.path.join(folder2, file)
        shutil.copy2(src, dst)
        print(f"Copied {file} from {folder1} to {folder2}")

    # Copy unique files from folder2 to folder1
    for file in unique_to_folder2:
        src = os.path.join(folder2, file)
        dst = os.path.join(folder1, file)
        shutil.copy2(src, dst)
        print(f"Copied {file} from {folder2} to {folder1}")

In [ ]:
source = '/content/drive/MyDrive/Dataset- -Conference/train_img'
destination = '/content/drive/MyDrive/Dataset- -Conference/train'

sync_folders(source, destination)

In [ ]:
source = '/content/drive/MyDrive/Dataset- -Conference/train_img'
destination = '/content/drive/MyDrive/Dataset- -Conference/train'

copy_folder_contents(source, destination)

In [ ]:
source = '/content/drive/MyDrive/Dataset- -Conference/val'
destination = '/content/drive/MyDrive/Dataset- -Conference/valid'

copy_folder_contents(source, destination)

In [ ]:
source = '/content/drive/MyDrive/Dataset- -Conference/test'
destination = '/content/drive/MyDrive/Dataset- -Conference/test/images'

copy_folder_contents(source, destination)

In [ ]:
import os

def compare_images_and_xmls(folder):
    # List of all files in the folder
    files = os.listdir(folder)

    # Filter out image files and XML files
    image_files = [f for f in files if f.endswith(('.jpg', '.jpeg', '.png'))]
    xml_files = [f for f in files if f.endswith('.xml')]

    # Get the base names of the files (without extensions)
    image_basenames = {os.path.splitext(img)[0] for img in image_files}
    xml_basenames = {os.path.splitext(xml)[0] for xml in xml_files}

    # Images that do not have corresponding XML files
    images_without_xml = image_basenames - xml_basenames

    # Count total images
    total_images = len(image_files)

    print(f"Total number of images: {total_images}")

    if images_without_xml:
        print("Images without corresponding XML files:")
        for img in images_without_xml:
            print(f"{img}.jpg (or .jpeg/.png)")
    else:
        print("All images have corresponding XML files.")

In [ ]:
# Replace 'folder_path' with the actual path to your folder
folder_path = '/content/drive/MyDrive/Dataset- -Conference/train/images'

compare_images_and_xmls(folder_path)

In [ ]:
%cd /content/drive/MyDrive/Dataset-\ -Conference/train

In [ ]:
%ls

In [ ]:
%mkdir labels
%mkdir images

In [ ]:
%mv *.png ./images/
%mv *.xml ./labels/

In [ ]:
%cd images

In [ ]:
%ls

In [ ]:
import os
import shutil

def organize_files(base_folder):
    # Define the paths for the new subfolders
    images_folder = os.path.join(base_folder, 'images')
    annotations_folder = os.path.join(base_folder, 'annotations')

    # Create the subfolders if they don't exist
    os.makedirs(images_folder, exist_ok=True)
    os.makedirs(annotations_folder, exist_ok=True)

    # List all files in the base folder
    files = os.listdir(base_folder)

    # Iterate over the files in the base folder
    for file in files:
        # Full path of the file
        file_path = os.path.join(base_folder, file)

        # Skip directories
        if os.path.isdir(file_path):
            continue

        # Move image files to the images folder
        if file.endswith(('.jpg', '.jpeg', '.png')):
            shutil.move(file_path, os.path.join(images_folder, file))
            print(f"Moved {file} to {images_folder}")

        # Move XML files to the annotations folder
        elif file.endswith('.xml'):
            shutil.move(file_path, os.path.join(annotations_folder, file))
            print(f"Moved {file} to {annotations_folder}")

In [ ]:
train_folder_path = '/content/drive/MyDrive/Dataset- -Conference/train'

organize_files(train_folder_path)

Moved V0169_375.png to /content/drive/MyDrive/Dataset- -Conference/train/images
Moved V0171_5210.png to /content/drive/MyDrive/Dataset- -Conference/train/images
Moved V0173_3700.png to /content/drive/MyDrive/Dataset- -Conference/train/images
Moved V0171_1800.png to /content/drive/MyDrive/Dataset- -Conference/train/images
Moved V0171_9410.png to /content/drive/MyDrive/Dataset- -Conference/train/images
Moved V0171_1920.png to /content/drive/MyDrive/Dataset- -Conference/train/images
Moved V0171_5590.png to /content/drive/MyDrive/Dataset- -Conference/train/images
Moved V0173_3710.png to /content/drive/MyDrive/Dataset- -Conference/train/images
Moved V0171_1810.png to /content/drive/MyDrive/Dataset- -Conference/train/images
Moved V0107_4400.png to /content/drive/MyDrive/Dataset- -Conference/train/images
Moved V0171_1910.png to /content/drive/MyDrive/Dataset- -Conference/train/images
Moved V0124_1480.png to /content/drive/MyDrive/Dataset- -Conference/train/images
Moved V0174_1440.png to /cont

In [ ]:
valid_folder_path = '/content/drive/MyDrive/Dataset- -Conference/valid'

organize_files(valid_folder_path)

Moved V0181_3640.xml to /content/drive/MyDrive/Dataset- -Conference/valid/annotations
Moved V0167_6380.xml to /content/drive/MyDrive/Dataset- -Conference/valid/annotations
Moved V0145_3440.png to /content/drive/MyDrive/Dataset- -Conference/valid/images
Moved V0181_3730.xml to /content/drive/MyDrive/Dataset- -Conference/valid/annotations
Moved V0181_3640.png to /content/drive/MyDrive/Dataset- -Conference/valid/images
Moved V0181_44860.xml to /content/drive/MyDrive/Dataset- -Conference/valid/annotations
Moved V0181_34270.xml to /content/drive/MyDrive/Dataset- -Conference/valid/annotations
Moved V0181_47410.png to /content/drive/MyDrive/Dataset- -Conference/valid/images
Moved V0167_6370.xml to /content/drive/MyDrive/Dataset- -Conference/valid/annotations
Moved V0145_3440.xml to /content/drive/MyDrive/Dataset- -Conference/valid/annotations
Moved V0181_47410.xml to /content/drive/MyDrive/Dataset- -Conference/valid/annotations
Moved V0181_3730.png to /content/drive/MyDrive/Dataset- -Conferen

In [ ]:
import os

def check_image_annotations(images_folder, annotations_folder):
    # List all image files in the images folder
    image_files = [f for f in os.listdir(images_folder) if f.endswith(('.jpg', '.jpeg', '.png'))]

    # List all XML files in the annotations folder
    xml_files = [f for f in os.listdir(annotations_folder) if f.endswith('.xml')]

    # Get the base names of the files (without extensions)
    image_basenames = {os.path.splitext(img)[0] for img in image_files}
    xml_basenames = {os.path.splitext(xml)[0] for xml in xml_files}

    # Find images without corresponding XML files
    images_without_xml = image_basenames - xml_basenames

    # Report results
    if images_without_xml:
        print("Images without corresponding XML files:")
        for img in images_without_xml:
            print(f"{img}.jpg (or .jpeg/.png)")
    else:
        print("All images have corresponding XML files.")

In [ ]:
# Replace 'images_folder_path' and 'annotations_folder_path' with the actual paths
train_images_folder_path = '/content/drive/MyDrive/Dataset- -Conference/train/images'
train_annotations_folder_path = '/content/drive/MyDrive/Dataset- -Conference/train/annotations'

check_image_annotations(train_images_folder_path, train_annotations_folder_path)

All images have corresponding XML files.


In [ ]:
# Replace 'images_folder_path' and 'annotations_folder_path' with the actual paths
valid_images_folder_path = '/content/drive/MyDrive/Dataset- -Conference/valid/images'
valid_annotations_folder_path = '/content/drive/MyDrive/Dataset- -Conference/valid/annotations'

check_image_annotations(valid_images_folder_path, valid_annotations_folder_path)

All images have corresponding XML files.


In [ ]:
# Replace 'images_folder_path' and 'annotations_folder_path' with the actual paths
test_images_folder_path = '/content/drive/MyDrive/Dataset- -Conference/test/images'
test_annotations_folder_path = '/content/drive/MyDrive/Dataset- -Conference/test/annotation'

check_image_annotations(valid_images_folder_path, valid_annotations_folder_path)

All images have corresponding XML files.


In [ ]:
# create Ultralytics specific text file of training images
file = open("train_images_conference.txt", "w")
for root, dirs, files in os.walk("."):
    for filename in files:
      # print("../train/images/" + filename)
      if filename == "train_images_conference.txt":
        pass
      else:
        file.write("../train/images/" + filename + "\n")
file.close()

In [ ]:
%cat train_images_conference.txt

../train/images/V0031_198.png
../train/images/V0031_210.png
../train/images/V0031_263.png
../train/images/V0043_1840.png
../train/images/V0043_2390.png
../train/images/V0043_2400.png
../train/images/V0031_270.png
../train/images/V0046_3940.png
../train/images/V0031_285.png
../train/images/V0031_292.png
../train/images/V0031_297.png
../train/images/V0031_310.png
../train/images/V0031_315.png
../train/images/V0031_325.png
../train/images/V0031_329.png
../train/images/V0043_100.png
../train/images/V0043_1180.png
../train/images/V0043_1190.png
../train/images/V0043_1850.png
../train/images/V0043_80.png
../train/images/V0046_3270.png
../train/images/V0046_3280.png
../train/images/V0046_4760.png
../train/images/V0046_4770.png
../train/images/V0046_5560.png
../train/images/V0046_5570.png
../train/images/V0047_1240.png
../train/images/V0047_1270.png
../train/images/V0047_1290.png
../train/images/V0047_1360.png
../train/images/V0047_620.png
../train/images/V0047_630.png
../train/images/V0047_88

In [ ]:
%cd /content/drive/MyDrive/Dataset-\ -Conference/val

/content/drive/MyDrive/Dataset- -Conference/val


In [ ]:
%mkdir labels
%mkdir images

In [ ]:
%mv *.png ./images/
!cp annotation/* labels/

mv: cannot stat '*.png': No such file or directory


In [ ]:
%cd images

/content/drive/MyDrive/Dataset- -Conference/val/images


In [ ]:
!pip install pylabel

  Using cached jedi-0.19.1-py2.py3-none-any.whl.metadata (22 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 213.7/213.7 kB 20.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 727.4/727.4 kB 45.8 MB/s eta 0:00:00
Using cached jedi-0.19.1-py2.py3-none-any.whl (1.6 MB)


In [ ]:
from pylabel import importer
dataset = importer.ImportVOC(path='/content/drive/MyDrive/Dataset- -Conference/train/annotations')
dataset.export.ExportToYoloV5()

Exporting YOLO files...: 100%|██████████| 2096/2096 [00:05<00:00, 378.14it/s]


['training/dataset.yaml',
 'training/labels/V0049_10770.txt',
 'training/labels/V0107_4190.txt',
 'training/labels/V0110_1450.txt',
 'training/labels/V0108_630.txt',
 'training/labels/V0107_2250.txt',
 'training/labels/V0107_4220.txt',
 'training/labels/V0107_3000.txt',
 'training/labels/V0107_3130.txt',
 'training/labels/V0088_130.txt',
 'training/labels/V0083_60.txt',
 'training/labels/V0107_3260.txt',
 'training/labels/V0088_200.txt',
 'training/labels/V0037_326.txt',
 'training/labels/V0108_540.txt',
 'training/labels/V0107_3350.txt',
 'training/labels/V0170_80.txt',
 'training/labels/V0102_250.txt',
 'training/labels/V0102_290.txt',
 'training/labels/V0171_10280.txt',
 'training/labels/V0046_2330.txt',
 'training/labels/V0107_2500.txt',
 'training/labels/V0107_40.txt',
 'training/labels/V0107_2170.txt',
 'training/labels/V0083_30.txt',
 'training/labels/V0171_7910.txt',
 'training/labels/V0175_1730.txt',
 'training/labels/V0105_210.txt',
 'training/labels/V0107_3300.txt',
 'traini

In [ ]:
import shutil
import os

def copy_folder(source_folder, destination_folder):
    # Ensure the destination folder exists; if not, create it
    os.makedirs(destination_folder, exist_ok=True)

    # Copy the source folder and its contents to the destination folder
    shutil.copytree(source_folder, destination_folder, dirs_exist_ok=True)

    print(f"Copied '{source_folder}' to '{destination_folder}'")

In [ ]:
# Replace 'source_folder_path' and 'destination_folder_path' with the actual paths
source_folder_path = '/content/training/labels'
destination_folder_path = '/content/drive/MyDrive/Dataset- -Conference/train'

copy_folder(source_folder_path, destination_folder_path)

Copied '/content/training/labels' to '/content/drive/MyDrive/Dataset- -Conference/train'


In [ ]:
import shutil
import os

def copy_file(source_file, destination_folder):
    # Ensure the destination folder exists; if not, create it
    os.makedirs(destination_folder, exist_ok=True)

    # Define the destination path for the file
    destination_path = os.path.join(destination_folder, os.path.basename(source_file))

    # Copy the file to the destination folder
    shutil.copy2(source_file, destination_path)

    print(f"Copied '{source_file}' to '{destination_folder}'")

In [ ]:
source = '/content/training/dataset.yaml'
destination = '/content/drive/MyDrive/Dataset- -Conference/train'
copy_file(source, destination)

Copied '/content/training/dataset.yaml' to '/content/drive/MyDrive/Dataset- -Conference/train'


In [ ]:
import os
import shutil

def organize_text_files(source_folder):
    # Define the path for the labels folder
    labels_folder = os.path.join(source_folder, 'labels')

    # Create the labels folder if it doesn't exist
    os.makedirs(labels_folder, exist_ok=True)

    # List all files in the source folder
    files = os.listdir(source_folder)

    # Iterate over all files in the source folder
    for file in files:
        # Check if the file is a text file (ends with .txt)
        if file.endswith('.txt'):
            # Construct the full file paths
            source_path = os.path.join(source_folder, file)
            destination_path = os.path.join(labels_folder, file)

            # Move the file to the labels folder
            shutil.move(source_path, destination_path)
            print(f"Moved '{file}' to '{labels_folder}'")

In [ ]:
source_folder_path = '/content/drive/MyDrive/Dataset- -Conference/train'

organize_text_files(source_folder_path)

Moved 'V0046_4770.txt' to '/content/drive/MyDrive/Dataset- -Conference/train/labels'
Moved 'V0065_130.txt' to '/content/drive/MyDrive/Dataset- -Conference/train/labels'
Moved 'V0181_780.txt' to '/content/drive/MyDrive/Dataset- -Conference/train/labels'
Moved 'V0181_22910.txt' to '/content/drive/MyDrive/Dataset- -Conference/train/labels'
Moved 'V0078_130.txt' to '/content/drive/MyDrive/Dataset- -Conference/train/labels'
Moved 'V0171_8690.txt' to '/content/drive/MyDrive/Dataset- -Conference/train/labels'
Moved 'V0146_1860.txt' to '/content/drive/MyDrive/Dataset- -Conference/train/labels'
Moved 'V0168_1410.txt' to '/content/drive/MyDrive/Dataset- -Conference/train/labels'
Moved 'V0050_3250.txt' to '/content/drive/MyDrive/Dataset- -Conference/train/labels'
Moved 'V0049_2250.txt' to '/content/drive/MyDrive/Dataset- -Conference/train/labels'
Moved 'V0113_60.txt' to '/content/drive/MyDrive/Dataset- -Conference/train/labels'
Moved 'V0108_630.txt' to '/content/drive/MyDrive/Dataset- -Conference

In [ ]:
import os

def count_text_files(folder_path):
    # List all files in the folder
    files = os.listdir(folder_path)

    # Filter the list to only include .txt files
    text_files = [file for file in files if file.endswith('.txt')]

    # Count the number of .txt files
    text_file_count = len(text_files)

    print(f"Number of text files in '{folder_path}': {text_file_count}")

    return text_file_count

In [ ]:
# Replace 'folder_path' with the actual path to your folder
folder_path = '/content/drive/MyDrive/Dataset- -Conference/train/labels'

count_text_files(folder_path)

Number of text files in '/content/drive/MyDrive/Dataset- -Conference/train/labels': 2096


2096

In [9]:
import os
import shutil

def empty_folder(folder_path):
    # List all files and subdirectories in the folder
    for filename in os.listdir(folder_path):
        file_path = os.path.join(folder_path, filename)

        # Check if it's a file or directory
        if os.path.isfile(file_path) or os.path.islink(file_path):
            os.unlink(file_path)  # Remove the file or link
            print(f"Deleted file: {file_path}")
        elif os.path.isdir(file_path):
            shutil.rmtree(file_path)  # Remove the directory and its contents
            print(f"Deleted directory: {file_path}")

In [ ]:
# Replace 'folder_path' with the actual path to your folder
folder_path = '/content/training'

empty_folder(folder_path)

Deleted directory: /content/training/labels
Deleted file: /content/training/dataset.yaml


In [ ]:
from pylabel import importer
dataset = importer.ImportVOC(path='/content/drive/MyDrive/Dataset- -Conference/valid/annotations')
dataset.export.ExportToYoloV5()

Exporting YOLO files...: 100%|██████████| 256/256 [00:00<00:00, 632.51it/s]


['training/dataset.yaml',
 'training/labels/V0181_3640.txt',
 'training/labels/V0167_6380.txt',
 'training/labels/V0181_3730.txt',
 'training/labels/V0181_44860.txt',
 'training/labels/V0181_34270.txt',
 'training/labels/V0167_6370.txt',
 'training/labels/V0145_3440.txt',
 'training/labels/V0181_47410.txt',
 'training/labels/V0181_41880.txt',
 'training/labels/V0153_170.txt',
 'training/labels/V0113_150.txt',
 'training/labels/V0151_40.txt',
 'training/labels/V0142_200.txt',
 'training/labels/V0142_240.txt',
 'training/labels/V0168_3150.txt',
 'training/labels/V0112_410.txt',
 'training/labels/V0151_60.txt',
 'training/labels/V0051_4750.txt',
 'training/labels/V0166_3700.txt',
 'training/labels/V0142_185.txt',
 'training/labels/V0050_2130.txt',
 'training/labels/V0053_40.txt',
 'training/labels/V0167_810.txt',
 'training/labels/V0051_2550.txt',
 'training/labels/V0051_2520.txt',
 'training/labels/V0046_4790.txt',
 'training/labels/V0166_2680.txt',
 'training/labels/V0046_3260.txt',
 't

In [ ]:
# Replace 'source_folder_path' and 'destination_folder_path' with the actual paths
valid_source_folder_path = '/content/training/labels'
valid_destination_folder_path = '/content/drive/MyDrive/Dataset- -Conference/valid'

copy_folder(valid_source_folder_path, valid_destination_folder_path)

Copied '/content/training/labels' to '/content/drive/MyDrive/Dataset- -Conference/valid'


In [ ]:
valid_source_folder_path = '/content/drive/MyDrive/Dataset- -Conference/valid'

organize_text_files(valid_source_folder_path)

Moved 'V0181_22890.txt' to '/content/drive/MyDrive/Dataset- -Conference/valid/labels'
Moved 'V0171_9510.txt' to '/content/drive/MyDrive/Dataset- -Conference/valid/labels'
Moved 'V0181_23250.txt' to '/content/drive/MyDrive/Dataset- -Conference/valid/labels'
Moved 'V0181_48200.txt' to '/content/drive/MyDrive/Dataset- -Conference/valid/labels'
Moved 'V0108_290.txt' to '/content/drive/MyDrive/Dataset- -Conference/valid/labels'
Moved 'V0181_45030.txt' to '/content/drive/MyDrive/Dataset- -Conference/valid/labels'
Moved 'V0048_1280.txt' to '/content/drive/MyDrive/Dataset- -Conference/valid/labels'
Moved 'V0169_460.txt' to '/content/drive/MyDrive/Dataset- -Conference/valid/labels'
Moved 'V0168_2020.txt' to '/content/drive/MyDrive/Dataset- -Conference/valid/labels'
Moved 'V0107_4200.txt' to '/content/drive/MyDrive/Dataset- -Conference/valid/labels'
Moved 'V0171_9490.txt' to '/content/drive/MyDrive/Dataset- -Conference/valid/labels'
Moved 'V0049_11200.txt' to '/content/drive/MyDrive/Dataset- -Co

In [ ]:
count_text_files('/content/drive/MyDrive/Dataset- -Conference/valid/labels')

Number of text files in '/content/drive/MyDrive/Dataset- -Conference/valid/labels': 256


256

In [ ]:
# Replace 'folder_path' with the actual path to your folder
folder_path = '/content/training'

empty_folder(folder_path)

Deleted directory: /content/training/labels
Deleted file: /content/training/dataset.yaml


In [ ]:
from pylabel import importer
dataset = importer.ImportVOC(path='/content/drive/MyDrive/Dataset- -Conference/test/annotation')
dataset.export.ExportToYoloV5()

Exporting YOLO files...: 100%|██████████| 662/662 [00:01<00:00, 621.54it/s]


['training/dataset.yaml',
 'training/labels/V0170_580.txt',
 'training/labels/V0170_560.txt',
 'training/labels/V0170_570.txt',
 'training/labels/V0107_4330.txt',
 'training/labels/V0107_4780.txt',
 'training/labels/V0102_120.txt',
 'training/labels/V0102_220.txt',
 'training/labels/V0181_1920.txt',
 'training/labels/V0107_4800.txt',
 'training/labels/V0102_210.txt',
 'training/labels/V0107_4740.txt',
 'training/labels/V0107_4760.txt',
 'training/labels/V0107_4820.txt',
 'training/labels/V0102_10.txt',
 'training/labels/V0102_50.txt',
 'training/labels/V0102_70.txt',
 'training/labels/V0093_130.txt',
 'training/labels/V0093_180.txt',
 'training/labels/V0102_200.txt',
 'training/labels/V0093_150.txt',
 'training/labels/V0102_130.txt',
 'training/labels/V0102_90.txt',
 'training/labels/V0093_160.txt',
 'training/labels/V0093_10.txt',
 'training/labels/V0093_100.txt',
 'training/labels/V0093_20.txt',
 'training/labels/V0093_90.txt',
 'training/labels/V0102_30.txt',
 'training/labels/V0093

In [ ]:
# Replace 'source_folder_path' and 'destination_folder_path' with the actual paths
test_source_folder_path = '/content/training/labels'
test_destination_folder_path = '/content/drive/MyDrive/Dataset- -Conference/test'

copy_folder(test_source_folder_path, test_destination_folder_path)

Copied '/content/training/labels' to '/content/drive/MyDrive/Dataset- -Conference/test'


In [ ]:
test_source_folder_path = '/content/drive/MyDrive/Dataset- -Conference/test'

organize_text_files(test_source_folder_path)

Moved 'V0111_50.txt' to '/content/drive/MyDrive/Dataset- -Conference/test/labels'
Moved 'V0056_100.txt' to '/content/drive/MyDrive/Dataset- -Conference/test/labels'
Moved 'V0141_285.txt' to '/content/drive/MyDrive/Dataset- -Conference/test/labels'
Moved 'V0167_13430.txt' to '/content/drive/MyDrive/Dataset- -Conference/test/labels'
Moved 'V0049_6170.txt' to '/content/drive/MyDrive/Dataset- -Conference/test/labels'
Moved 'V0049_650.txt' to '/content/drive/MyDrive/Dataset- -Conference/test/labels'
Moved 'V0112_160.txt' to '/content/drive/MyDrive/Dataset- -Conference/test/labels'
Moved 'V0181_2230.txt' to '/content/drive/MyDrive/Dataset- -Conference/test/labels'
Moved 'V0048_4040.txt' to '/content/drive/MyDrive/Dataset- -Conference/test/labels'
Moved 'V0046_2170.txt' to '/content/drive/MyDrive/Dataset- -Conference/test/labels'
Moved 'V0146_2785.txt' to '/content/drive/MyDrive/Dataset- -Conference/test/labels'
Moved 'V0079_690.txt' to '/content/drive/MyDrive/Dataset- -Conference/test/labels

In [ ]:
count_text_files('/content/drive/MyDrive/Dataset- -Conference/test/labels')

Number of text files in '/content/drive/MyDrive/Dataset- -Conference/test/labels': 662


662

In [ ]:
# Replace 'folder_path' with the actual path to your folder
folder_path = '/content/drive/MyDrive/Dataset- -Conference/test/images'

empty_folder(folder_path)

Deleted file: /content/drive/MyDrive/Dataset- -Conference/test/images/V0181_49750.png
Deleted file: /content/drive/MyDrive/Dataset- -Conference/test/images/V0181_49090.png
Deleted file: /content/drive/MyDrive/Dataset- -Conference/test/images/V0181_49110.png
Deleted file: /content/drive/MyDrive/Dataset- -Conference/test/images/V0181_48260.png
Deleted file: /content/drive/MyDrive/Dataset- -Conference/test/images/V0181_49770.png
Deleted file: /content/drive/MyDrive/Dataset- -Conference/test/images/V0181_47550.png
Deleted file: /content/drive/MyDrive/Dataset- -Conference/test/images/V0181_49760.png
Deleted file: /content/drive/MyDrive/Dataset- -Conference/test/images/V0181_49100.png
Deleted file: /content/drive/MyDrive/Dataset- -Conference/test/images/V0181_48270.png
Deleted file: /content/drive/MyDrive/Dataset- -Conference/test/images/V0181_49780.png
Deleted file: /content/drive/MyDrive/Dataset- -Conference/test/images/V0181_48280.png
Deleted file: /content/drive/MyDrive/Dataset- -Confere

In [ ]:
import os
import shutil

def organize_images(base_folder):
    # Define the path for the images folder
    images_folder = os.path.join(base_folder, 'images')

    # Create the images folder if it doesn't exist
    os.makedirs(images_folder, exist_ok=True)

    # List all files in the base folder
    files = os.listdir(base_folder)

    # Iterate over all files in the base folder
    for file in files:
        # Construct the full path of the file
        file_path = os.path.join(base_folder, file)

        # Check if the file is an image file (with common image extensions)
        if os.path.isfile(file_path) and file.lower().endswith(('.jpg', '.jpeg', '.png', '.gif', '.bmp')):
            # Move the image file to the images folder
            shutil.move(file_path, os.path.join(images_folder, file))
            print(f"Moved '{file}' to '{images_folder}'")

In [ ]:
base_folder_path = '/content/drive/MyDrive/Dataset- -Conference/test'

organize_images(base_folder_path)

Moved 'V0181_49750.png' to '/content/drive/MyDrive/Dataset- -Conference/test/images'
Moved 'V0181_49090.png' to '/content/drive/MyDrive/Dataset- -Conference/test/images'
Moved 'V0181_49110.png' to '/content/drive/MyDrive/Dataset- -Conference/test/images'
Moved 'V0181_48260.png' to '/content/drive/MyDrive/Dataset- -Conference/test/images'
Moved 'V0181_49770.png' to '/content/drive/MyDrive/Dataset- -Conference/test/images'
Moved 'V0181_47550.png' to '/content/drive/MyDrive/Dataset- -Conference/test/images'
Moved 'V0181_49760.png' to '/content/drive/MyDrive/Dataset- -Conference/test/images'
Moved 'V0181_49100.png' to '/content/drive/MyDrive/Dataset- -Conference/test/images'
Moved 'V0181_48270.png' to '/content/drive/MyDrive/Dataset- -Conference/test/images'
Moved 'V0181_49780.png' to '/content/drive/MyDrive/Dataset- -Conference/test/images'
Moved 'V0181_48280.png' to '/content/drive/MyDrive/Dataset- -Conference/test/images'
Moved 'V0181_47530.png' to '/content/drive/MyDrive/Dataset- -Conf

In [ ]:
import os

def count_images(folder_path):
    # List all files in the folder
    files = os.listdir(folder_path)

    # Filter the list to include only image files
    image_files = [file for file in files if file.lower().endswith(('.jpg', '.jpeg', '.png', '.gif', '.bmp'))]

    # Count the number of image files
    image_count = len(image_files)

    print(f"Number of image files in '{folder_path}': {image_count}")

    return image_count

In [ ]:
# Replace 'folder_path' with the actual path to your folder
folder_path = '/content/drive/MyDrive/Dataset- -Conference/test/images'

count_images(folder_path)

Number of image files in '/content/drive/MyDrive/Dataset- -Conference/test/images': 662


662

In [ ]:
source = '/content/drive/MyDrive/Dataset- -Conference/train/dataset.yaml'
destination = '/content/drive/MyDrive/Dataset- -Conference'
copy_file(source, destination)

Copied '/content/drive/MyDrive/Dataset- -Conference/train/dataset.yaml' to '/content/drive/MyDrive/Dataset- -Conference'


In [ ]:
# create Ultralytics specific text file of train images
file = open("val_images_conference.txt", "w")
for root, dirs, files in os.walk("."):
    for filename in files:
      # print("../val/images/" + filename)
      if filename == "val_images_conference.txt":
        pass
      else:
        file.write("../val/images/" + filename + "\n")
file.close()

In [ ]:
%cat val_images_conference.txt

../val/images/V0181_34270.png
../val/images/V0181_3640.png
../val/images/V0181_3730.png
../val/images/V0181_44860.png
../val/images/V0168_3150.png
../val/images/V0142_240.png
../val/images/V0181_47410.png
../val/images/V0145_3440.png
../val/images/V0181_41880.png
../val/images/V0167_6370.png
../val/images/V0113_150.png
../val/images/V0153_220.png
../val/images/V0051_4750.png
../val/images/V0050_2130.png
../val/images/V0142_185.png
../val/images/V0167_6380.png
../val/images/V0167_810.png
../val/images/V0151_60.png
../val/images/V0046_3260.png
../val/images/V0112_410.png
../val/images/V0151_40.png
../val/images/V0142_200.png
../val/images/V0051_2520.png
../val/images/V0053_40.png
../val/images/V0153_170.png
../val/images/V0051_2550.png
../val/images/V0166_3700.png
../val/images/V0049_3150.png
../val/images/V0145_1135.png
../val/images/V0181_3380.png
../val/images/V0166_5100.png
../val/images/V0051_5210.png
../val/images/V0168_2020.png
../val/images/V0166_1890.png
../val/images/V0166_2680

In [ ]:
%cd /content/yolov3/data

/content/yolov3/data


In [ ]:
%cd /content/drive/MyDrive/Dataset-\ -Conference

/content/drive/MyDrive/Dataset- -Conference


In [ ]:
%cat classNames.txt

SpeedLimit60kmh
School
PEDESTRIANCROSSING
SharpBendToTheLeft
UTurn
speedlimit80kmh
NoOvertaking
SharpBendToTheRight
SideRoadLeft
SideRoadRight
speedlimit40kmh
NarrowBridge
RoadHump
nouseofhorn
Crossroads

In [ ]:
!cp classNames.txt classNames_conference.names

In [ ]:
def get_num_classes(labels_file_path):
    classes = 0
    with open(labels_file_path, 'r') as f:
      for line in f:
        classes += 1
    return classes

In [ ]:
import os


# Check if the conference.data file exists
if os.path.isfile("conference.data"):
    # Open the existing file in read mode
    with open("conference.data", 'r') as f:
        lines = f.readlines()
else:
    # If the file doesn't exist, create it with a default value for the number of classes
    num_classes = 0  # Placeholder for the number of classes
    with open("classNames.txt", "r") as class_names_file:
        num_classes = sum(1 for _ in class_names_file)
    lines = ["classes={}\n".format(num_classes)]

# Add other lines to the list
lines.append("train=/content/drive/MyDrive/Dataset-\ -Conference/train/images/train_images_conference.txt\n")
lines.append("valid=/content/drive/MyDrive/Dataset-\ -Conference/val/images/val_images_conference.txt\n")
lines.append("names=/content/drive/MyDrive/Dataset-\ -Conference/classNames_conference.names\n")

# Write the modified content back to the file
with open("conference.data", 'w') as f:
    f.writelines(lines)


In [ ]:
%cat conference.data

classes=15
train=/content/drive/MyDrive/Dataset-\ -Conference/train/images/train_images_conference.txt
valid=/content/drive/MyDrive/Dataset-\ -Conference/val/images/val_images_conference.txt
names=/content/drive/MyDrive/Dataset-\ -Conference/classNames_conference.names


In [ ]:
%cat /content/drive/MyDrive/Dataset-\ -Conference/conference.data

classes=15
train=/content/drive/MyDrive/Dataset-\ -Conference/train/images/train_images_conference.txt
valid=/content/drive/MyDrive/Dataset-\ -Conference/val/images/val_images_conference.txt
names=/content/drive/MyDrive/Dataset-\ -Conference/classNames_conference.names


In [ ]:
%cd /content/drive/MyDrive/Dataset- -Conference

/content/drive/MyDrive/Dataset- -Conference


In [ ]:
%ls

bdtsd_fewx_train.json  classNames_conference.names  results/  test/       train_img.cache
bdtsd_train.json       classNames.txt               runs/     train/      val/
bdtsd_val.json         conference.data              split/    train_img/


In [ ]:
%cp /content/drive/MyDrive/Dataset-\ -Conference/conference.data /content/drive/MyDrive/conference.data

In [ ]:
%cd ..
import yaml

# Define the data
data = {
    'nc': 15,
    'train': '/content/drive/MyDrive/Dataset- -Conference/train/',
    'val': '/content/drive/MyDrive/Dataset- -Conference/val/',
    'names': [
        'SpeedLimit60kmh',
        'School',
        'PEDESTRIANCROSSING',
        'SharpBendToTheLeft',
        'UTurn',
        'speedlimit80kmh',
        'NoOvertaking',
        'SharpBendToTheRight',
        'SideRoadLeft',
        'SideRoadRight',
        'speedlimit40kmh',
        'NarrowBridge',
        'RoadHump',
        'nouseofhorn',
        'Crossroads'
    ]
}

# Write the data to conference.yml
with open('conference.yml', 'w') as yaml_file:
    yaml.dump(data, yaml_file)


/content/drive/MyDrive


In [5]:
%cd /content/yolov5

/content/yolov5


In [7]:
!python train.py --epochs 10 --data '/content/drive/MyDrive/Dataset- -Conference/dataset.yaml' --weights '/content/drive/MyDrive/Dataset- -Conference/yolov5/train/exp3/weights/last.pt'

2024-09-04 09:28:51.778579: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-09-04 09:28:51.798619: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-09-04 09:28:51.804612: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
train: weights=/content/drive/MyDrive/Dataset- -Conference/yolov5/train/exp3/weights/last.pt, cfg=, data=/content/drive/MyDrive/Dataset- -Conference/dataset.yaml, hyp=data/hyps/hyp.scratch-low.yaml, epochs=10, batch_size=16, imgsz=640, rect=False, resume=False, nosave=False, noval=False, noautoanchor=False, noplots=False, evolve=None, evolve_population=data/hyps,

In [12]:
empty_folder('/content/drive/MyDrive/Dataset- -Conference/yolov5_epoch16_35')

Deleted directory: /content/drive/MyDrive/Dataset- -Conference/yolov5_epoch16_35/train


In [14]:
import shutil
source_folder = '/content/yolov5/runs'
destination_folder = '/content/drive/MyDrive/Dataset- -Conference/yolov5/runs'  # Path to the destination folder
# Copy the folder and its contents to the destination
shutil.copytree(source_folder, destination_folder)

'/content/drive/MyDrive/Dataset- -Conference/yolov5/runs'

In [15]:
!python detect.py --weights /content/yolov5/runs/train/exp2/weights/best.pt --conf 0.1 --source '/content/drive/MyDrive/Dataset- -Conference/test/images'

detect: weights=['/content/yolov5/runs/train/exp2/weights/best.pt'], source=/content/drive/MyDrive/Dataset- -Conference/test/images, data=data/coco128.yaml, imgsz=[640, 640], conf_thres=0.1, iou_thres=0.45, max_det=1000, device=, view_img=False, save_txt=False, save_format=0, save_csv=False, save_conf=False, save_crop=False, nosave=False, classes=None, agnostic_nms=False, augment=False, visualize=False, update=False, project=runs/detect, name=exp, exist_ok=False, line_thickness=3, hide_labels=False, hide_conf=False, half=False, dnn=False, vid_stride=1
YOLOv5 🚀 v7.0-364-g4d39325f Python-3.10.12 torch-2.4.0+cu121 CUDA:0 (Tesla T4, 15102MiB)

Fusing layers... 
Model summary: 157 layers, 7050580 parameters, 0 gradients, 15.9 GFLOPs
WARNING ⚠️ NMS time limit 0.550s exceeded
image 1/662 /content/drive/MyDrive/Dataset- -Conference/test/images/V0029_0.png: 640x384 1 School, 51.6ms
image 2/662 /content/drive/MyDrive/Dataset- -Conference/test/images/V0029_13.png: 640x384 1 School, 7.6ms
image 3/

In [16]:
import shutil
source_folder = '/content/yolov5/runs'
destination_folder = '/content/drive/MyDrive/Dataset- -Conference/yolov5/finals'  # Path to the destination folder
# Copy the folder and its contents to the destination
shutil.copytree(source_folder, destination_folder)

'/content/drive/MyDrive/Dataset- -Conference/yolov5/finals'

In [ ]:
!python val.py --weights /content/drive/MyDrive/BD_Road_Sign_Dataset/yolov5/train/exp/weights/best.pt --data '/content/drive/MyDrive/BD_Road_Sign_Dataset/data.yaml' --img 640 --half

In [ ]:
import shutil
source_folder = '/content/yolov5/runs'
destination_folder = '/content/drive/MyDrive/BD_Road_Sign_Dataset/yolov5/final_val'  # Path to the destination folder
# Copy the folder and its contents to the destination
shutil.copytree(source_folder, destination_folder)